In [ ]:
!pip install langchain langchain_core langchain_text_splitters langchain_community wikipedia chromadb

# Install Ollama and download the LLM model and embeddding model

In [2]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [3]:
!nohup ollama serve &
!sleep 5 && tail /content/nohup.out

nohup: appending output to 'nohup.out'
time=2024-09-28T16:06:36.615Z level=INFO source=download.go:175 msg="downloading 970aa74c0a90 in 3 100 MB part(s)"
time=2024-09-28T16:06:41.209Z level=INFO source=download.go:175 msg="downloading c71d239df917 in 1 11 KB part(s)"
time=2024-09-28T16:06:43.752Z level=INFO source=download.go:175 msg="downloading ce4a164fc046 in 1 17 B part(s)"
time=2024-09-28T16:06:46.366Z level=INFO source=download.go:175 msg="downloading 31df23ea7daa in 1 420 B part(s)"
[GIN] 2024/09/28 - 16:06:48 | 200 | 14.495674682s |       127.0.0.1 | POST     "/api/pull"
2024/09/28 16:08:25 routes.go:1153: INFO server config env="map[CUDA_VISIBLE_DEVICES: GPU_DEVICE_ORDINAL: HIP_VISIBLE_DEVICES: HSA_OVERRIDE_GFX_VERSION: HTTPS_PROXY: HTTP_PROXY: NO_PROXY: OLLAMA_DEBUG:false OLLAMA_FLASH_ATTENTION:false OLLAMA_GPU_OVERHEAD:0 OLLAMA_HOST:http://127.0.0.1:11434 OLLAMA_INTEL_GPU:false OLLAMA_KEEP_ALIVE:5m0s OLLAMA_LLM_LIBRARY: OLLAMA_LOAD_TIMEOUT:5m0s OLLAMA_MAX_LOADED_MODELS:0 OLL

In [4]:
! ollama pull llama3.2

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest 
pulling dde5aa3fc5ff... 100% ▕▏ 2.0 GB                         
pulling 966de95ca8a6... 100% ▕▏ 1.4 KB                         
pulling fcc5a6bec9da... 100% ▕▏ 7.7 KB                         
pulling a70ff7e570d9... 100% ▕▏ 6.0 KB                         
pulling 56bb8bd477a5... 100% ▕▏   96 B                         
pulling 34bb5ab01051... 100% ▕▏  561 B                         
verifying sha256 digest 
writing manifest 
success 


In [5]:
! ollama pull nomic-embed-text

pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠸ pulling manifest ⠴ pulling manifest ⠴ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest 
pulling 970aa74c0a90... 100% ▕▏ 274 MB                         
pulling c71d239df917... 100% ▕▏  11 KB                         
pulling ce4a164fc046... 100% ▕▏   17 B                         
pulling 31df23ea7daa... 100% ▕▏  420 B                         
verifying sha256 digest 
writing manifest 
success 


In [6]:
from langchain.text_splitter import NLTKTextSplitter
from langchain.document_loaders import WikipediaLoader
import nltk
from langchain import PromptTemplate
from langchain.chains.question_answering import load_qa_chain
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [7]:
from langchain_community.llms import Ollama
llm = Ollama(model="llama3.2") # Llama 3.2 3B

In [8]:
query = "Isaac Newton"

# ==============
docs = WikipediaLoader(query=query, load_max_docs=1,
                         doc_content_chars_max=50_0000).load()

text_document = docs[0].page_content

In [9]:
documents = [ text_document]
metadatas = [{"document":query}]

In [10]:
text_splitter = NLTKTextSplitter(
    chunk_size=4096,
    chunk_overlap=32
) #This method splits the input documents into chunks , ensuring that there is an overlap of 32 characters between adjacent chunks.
tokens_chunks = text_splitter.create_documents(
    documents, metadatas=metadatas
)# passing the input documents and potentially the metadata for embedding model

**Create vector Database**


In [11]:
%%time
vector_db_Chroma = Chroma.from_documents(
                    documents=tokens_chunks,
                    embedding=OllamaEmbeddings(model="nomic-embed-text",show_progress=True), # embeding model
                    collection_name="rag",
)

OllamaEmbeddings: 100%|██████████| 13/13 [00:03<00:00,  3.54it/s]

CPU times: user 603 ms, sys: 107 ms, total: 711 ms
Wall time: 4.61 s


# Prepare Prompt Template

In [12]:
# Retrieve the similar docments using llm
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to generate three
    different versions of the given user question to retrieve relevant documents from
    a vector database. By generating multiple perspectives on the user question, your
    goal is to help the user overcome some of the limitations of the distance-based
    similarity search. Provide these alternative questions separated by newlines.
    Original question: {question}""",
)
retriever = MultiQueryRetriever.from_llm(
    vector_db_Chroma.as_retriever(),
    llm,
    prompt=QUERY_PROMPT
)

In [13]:
# RAG prompt
template = """Answer the question based ONLY on the following context:
{context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)
chain = prompt | llm

The pipeline takes a question as input. It retrieves relevant context using the retriever, formats the question and context using the prompt template, feeds the formatted prompt to the language model, and finally parses the model's response into a string.

In [14]:
full_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [15]:
def stream_output(chain,question):
    stream = chain.stream(question)
    for response in stream:
        print(response, end='')  # Streamed response piece by piece

In [20]:
# Example prompt
question = "In which year Newton die?"
stream_output(full_chain,question)

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00, 10.53it/s]


The question "In which year Newton die?" is incomplete, as it doesn't specify whether it's referring to Isaac Newton or another person named Newton. However, I'll provide an answer based on the context of the text.

Isaac Newton passed away in 1727.

In [17]:
question = "Was Newton ever marrid "
stream_output(full_chain,question)

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  7.47it/s]


No, according to the text, Isaac Newton was a bachelor, meaning he never married.

In [18]:
question = "tell me about gravity and summarize it into 3 points"
stream_output(full_chain,question)

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00, 18.35it/s]


Gravity is a fundamental force of nature that was first discovered by Sir Isaac Newton. Here's a summary of his work on gravity in three key points:

**Point 1: The Law of Universal Gravitation**
Newton formulated the Law of Universal Gravitation, which states that every point mass attracts every other point mass with a force proportional to the product of their masses and inversely proportional to the square of the distance between them. This law applies to all objects in the universe, regardless of their size or composition.

**Point 2: The Apple Incident**
According to Newton's own account, he was inspired to formulate his theory of gravity after observing an apple falling from a tree. He asked himself why the apple always fell perpendicularly to the ground, rather than going sideways or upwards. This simple observation led him to develop his law of universal gravitation.

**Point 3: Gravity is not just a force that pulls objects towards each other**
Newton's theory of gravity also 

# Another Way To retrive using similarity search

In [24]:
question = "Was Newton ever marrid "
similar_docs = vector_db_Chroma.similarity_search(question, k=5)
retriever = [ chunk.page_content for chunk in similar_docs ]
# Example prompt
#question = "In which year Newton die?"
stream_output(chain , {'context': retriever ,"question": question})

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00, 10.59it/s]


There is no mention of Newton being married in the provided text.